# Example stitching workflow

Using a 3x3 (t,c,y,z) dataset.

In [1]:
VISUALIZE_USING_NAPARI = True

In [2]:
import os, tempfile
import numpy as np
from pathlib import Path
from tqdm import tqdm
import dask.diagnostics
import dask.array as da

from multiview_stitcher import msi_utils
# %matplotlib notebook

In [3]:
# # Start a dask cluster

# from distributed import Client, LocalCluster

# lc = LocalCluster(n_workers=1, threads_per_worker=None)
# client = Client(lc)
# client

In [12]:
# Load dataset from file

from multiview_stitcher import io

filename = Path('/Users/malbert/software/multiview-stitcher/image-datasets/arthur_20220621_premovie_dish2-max.czi')
sims = io.read_mosaic_image_into_list_of_spatial_xarrays(filename, scene_index=0)

msims = [msi_utils.get_store_decorator(
    filename.with_suffix('.tile%03d.zarr' %isim), store_overwrite=False)(
        msi_utils.get_msim_from_sim)(sim, None)
    for isim, sim in tqdm(enumerate(sims))]

# msims = [msi_utils.get_msim_from_sim(sim, scale_factors=[]) for sim in sims]
print(f"Loaded {len(msims)} positions from {filename}.\nTransform key is {io.METADATA_TRANSFORM_KEY}.")

# consider only first tp
# msims = [msi_utils.multiscale_sel_coords(msim, {'t':0}) for msim in msims]


9it [00:00, 52.85it/s]

Loaded 9 positions from /Users/malbert/software/multiview-stitcher/image-datasets/arthur_20220621_premovie_dish2-max.czi.
Transform key is affine_metadata.


In [5]:
if VISUALIZE_USING_NAPARI:

    from napari_stitcher import viewer_utils
    import importlib
    viewer_utils = importlib.reload(viewer_utils)
    
    import napari
    viewer = napari.Viewer()
    
    layer_tuples = viewer_utils.create_image_layer_tuples_from_msims(
        msims, n_colors=2, transform_key='affine_metadata')
    
    viewer_utils.add_image_layer_tuples_to_viewer(viewer, layer_tuples)
    
    viewer.reset_view()

In [18]:
from multiview_stitcher import registration, fusion, mv_graph

import importlib
registration = importlib.reload(registration)
mv_graph = importlib.reload(mv_graph)

print('Registering...')
with dask.diagnostics.ProgressBar():
    params = registration.register(
        msims[:], transform_key='affine_metadata', reg_channel_index=0)

for msim, param in zip(msims, params):
    msi_utils.set_affine_transform(msim, param, transform_key='affine_registered', base_transform_key='affine_metadata')

Registering...
[########################################] | 100% Completed | 4.44 sms


In [19]:
import importlib
from multiview_stitcher import transformation
fusion = importlib.reload(fusion)
transformation = importlib.reload(transformation)

print('Creating fusion graph...')
with dask.diagnostics.ProgressBar():
    fused = fusion.fuse(
        [msi_utils.get_sim_from_msim(msim) for msim in msims],
        transform_key='affine_registered')#.compute()

print('Fusing views...')
tmpdir = tempfile.TemporaryDirectory()
with dask.diagnostics.ProgressBar():
    fused.data = da.to_zarr(
        fused.data,
        os.path.join(tmpdir.name, 'fused_sim.zarr'),
        overwrite=True, return_stored=True, compute=True)

print('Creating multiscale output OME-Zarr...')
with dask.diagnostics.ProgressBar():

    mfused = msi_utils.get_msim_from_sim(fused, scale_factors=None)

    fused_path = os.path.join(tmpdir.name, 'fused.zarr')
    mfused.to_zarr(fused_path)
    
mfused = msi_utils.multiscale_spatial_image_from_zarr(fused_path)

Creating fusion graph...
Fusing views...
[########################################] | 100% Completed | 12.03 s
Creating multiscale output OME-Zarr...
[########################################] | 100% Completed | 1.14 sms
[########################################] | 100% Completed | 2.73 sms
[########################################] | 100% Completed | 3.50 sms
[########################################] | 100% Completed | 3.61 sms


In [20]:
if VISUALIZE_USING_NAPARI:
    
    viewer = napari.Viewer(ndisplay=2)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_metadata', n_colors=2,
        name_prefix='pre-registered view',
        contrast_limits=(0, 1000),
        )
    mlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        [msi_utils.multiscale_sel_coords(msim, {'c': 'EGFP'}) for msim in msims],
        transform_key='affine_registered', n_colors=2,
        name_prefix='registered view',
        contrast_limits=(0, 1000),
        )
    mlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

    lds = viewer_utils.create_image_layer_tuples_from_msim(
        mfused,
        transform_key='affine_registered',
        name_prefix='fused',
        contrast_limits=[0, 1000])

    viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)



In [ ]:
# with dask.diagnostics.ProgressBar():
#     io.save_sim_as_tif('fused.tif', fused)